In [53]:
import warnings
warnings.filterwarnings('ignore')

import os
import csv
import numpy as np
import pandas as pd
import gymnasium as gym
import statsmodels.api as sm

import zipfile
from gymnasium import spaces
from datetime import date
from envs.env_gridsearch import kellycriterion
from sklearn.model_selection import train_test_split
from stable_baselines3.common.vec_env import DummyVecEnv
from utils.read2df import read2df

from itertools import combinations
from utils.cointcorr import CointnCorr

# os.makedirs("result/rl-restrict", exist_ok=True)

# for root, dirs, files in os.walk(f"result/rl-restrict/"):
#     for file in files:
#         os.remove(os.path.join(root, file))

## Data

In [ ]:
# symbols = ['BTCUSD', 'BTCGBP', 'BTCRUB', 'BTCEUR',]
symbols = ['BTCUSDT', 'BTCGBP', 'BTCRUB', 'BTCEUR']
start_date = '2023-01-01'
end_date = '2024-01-01'
trade_date = '2024-01-01'
folder = 'result'

freqs = {'1m' : 1}

In [ ]:
!python binance-public-data/python/download-kline.py \
        -s {" ".join(symbols)} -i {" ".join(list(freqs.keys()))} -startDate {start_date} -endDate {end_date} -t spot -skip-daily 1

Found 4 symbols
[1/4] - start download monthly BTCUSDT klines 

File Download: C:\Users\shaya\OneDrive\Documents\RLPairTradingProjectShayan\pair-trading-RLenvs\binance-public-data\python\data/spot/monthly/klines/BTCUSDT/1m/2023-01-01_2024-01-01\BTCUSDT-1m-2023-01.zip

[..................................................]
[..................................................]
[#.................................................]
[#.................................................]
[##................................................]
[##................................................]
[###...............................................]
[###...............................................]
[####..............................................]
[####..............................................]
[#####.............................................]
[#####.............................................]
[######............................................]
[######...................................

In [ ]:
# List of symbols to merge
symbols = ['BTCUSDT', 'BTCEUR', 'BTCRUB', 'BTCGBP']

# List to store individual DataFrames
mins = ['1m', '3m', '5m']

rawdfs = []
for min in mins:
    # Loop through each symbol
    for symbol in symbols:
        
        directory = f'./klines/{symbol}/{min}/{start_date}_{end_date}/'
        
        # Loop through each zip file in the directory
        count = 0 
        for file_name in os.listdir(directory):
            
            if file_name.endswith('.zip'):
                with zipfile.ZipFile(os.path.join(directory, file_name), 'r') as zip_ref:
                    # only one CSV file in each zip archive
                    csv_file = zip_ref.namelist()[0]
                    with zip_ref.open(csv_file) as csv_fp:
                        # Read the CSV data into a DataFrame
                        temp_df = pd.read_csv(csv_fp, header=None)
                        temp_df.columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
                        # temp_df['date'] = pd.to_datetime(temp_df['close_time'], unit='ms').dt.strftime('%Y-%m-%d')
                        temp_df = temp_df.rename(columns={"close_time": "time"})
                        temp_df['tic'] = symbol
                        temp_df['time_int'] = min
                        print(f"{symbol} - {min} = shape {temp_df.shape}")
                        rawdfs.append(temp_df[['time', 'open', 'high', 'low', 'close', 'volume', 'tic', 'time_int']])
                count += 1

        print(f"{symbol} - {min} = #{count} files")

# Concatenate all DataFrames into a single DataFrame
rawdf = pd.concat(rawdfs, ignore_index=True)

# Count the number of unique 'tic' values per date
tic_counts = rawdf.groupby('time')['tic'].nunique()

# Filter the DataFrame to keep only rows where all 'tic' values participate
df = rawdf[rawdf['time'].isin(tic_counts[tic_counts == len(rawdf['tic'].unique())].index)]
# Only wanted columns
df = df[['time', 'open', 'high', 'low', 'close', 'volume', 'tic', 'time_int']]
df['datetime'] = pd.to_datetime(df['time'], unit='ms')

df = df.sort_values(['time', 'tic', 'time_int'],ignore_index=True)

df

BTCUSDT - 1m = shape (44640, 14)
       open_time      open      high       low     close     volume  \
0  1672531200000  16541.77  16544.76  16538.45  16543.67   83.08143   
1  1672531260000  16543.04  16544.41  16538.48  16539.31   80.45300   
2  1672531320000  16539.31  16541.17  16534.52  16536.43   62.90197   
3  1672531380000  16536.43  16537.28  16531.00  16533.65  115.71894   
4  1672531440000  16534.12  16536.08  16527.51  16535.38  144.45369   

            time  quote_asset_volume  number_of_trades  \
0  1672531259999        1.374269e+06              2687   
1  1672531319999        1.330773e+06              2890   
2  1672531379999        1.040248e+06              1930   
3  1672531439999        1.913268e+06              2956   
4  1672531499999        2.388081e+06              3795   

   taker_buy_base_asset_volume  taker_buy_quote_asset_volume  ignore      tic  \
0                     40.18369                  6.647060e+05       0  BTCUSDT   
1                     33.2842

,time,open,high,low,close,volume,tic,time_int,datetime
0,1672531259999,15425.08,15426.91,15422.74,15426.08,0.72493,BTCEUR,1m,2023-01-01 00:00:59.999
1,1672531259999,13678.76,13678.76,13678.76,13678.76,0.01089,BTCGBP,1m,2023-01-01 00:00:59.999
2,1672531259999,1195551.00,1195551.00,1195551.00,1195551.00,0.00000,BTCRUB,1m,2023-01-01 00:00:59.999
3,1672531259999,16541.77,16544.76,16538.45,16543.67,83.08143,BTCUSDT,1m,2023-01-01 00:00:59.999
4,1672531319999,15425.24,15429.79,15423.96,15424.21,0.19929,BTCEUR,1m,2023-01-01 00:01:59.999
...,...,...,...,...,...,...,...,...,...
3197783,1703818799999,3725017.00,3726499.00,3722975.00,3722975.00,0.00210,BTCRUB,3m,2023-12-29 02:59:59.999
3197784,1703818799999,3726130.00,3726499.00,3722975.00,3722975.00,0.00620,BTCRUB,5m,2023-12-29 02:59:59.999
3197785,1703818799999,42561.81,42574.77,42561.80,42574.71,13.04876,BTCUSDT,1m,2023-12-29 02:59:59.999
3197786,1703818799999,42558.91,42574.77,42552.39,42574.71,39.72575,BTCUSDT,3m,2023-12-29 02:59:59.999


In [56]:
from params import *

'''
Set data before `trade_data` as training data, after `trade_data` is trade_data
'''

trains, tests = [], []
trains.append(df[(df['datetime'] > start_date) & (df['datetime'] < trade_date)].reset_index(drop=True))
tests.append(df[(df['datetime'] >= trade_date) & (df['datetime'] < end_date)].reset_index(drop=True))

In [59]:
trains = [df for _, df in trains[0].groupby('time_int')]
tests = [df for _, df in tests[0].groupby('time_int')]

In [69]:
'''
Find the best pair
'''

import pickle

tables = CointnCorr(trains, freqs).tabulate()
cointncorrtxt = f"result/gridsearch/cointncorr.txt"

os.remove(f"{cointncorrtxt}") if os.path.exists(f"{cointncorrtxt}") else None

with open(cointncorrtxt, "w+") as f:
    for k, v in tables.items():
        f.write(f"{k}\n")
        f.write(f"{v}\n\n")
f.close()

best_value = 0
for key in tables.keys():
    for freq in freqs:
        rel = tables[key].at['coint', freq]*3 + tables[key].at['corr', freq]
        if rel > best_value:
            best_value = rel
            best_pair = key
            best_freq = freq

print("===========================================")
print(f"Best trading pairs shall be: {best_pair} under {best_freq} interval")
print(f"the coint is {round(tables[best_pair].at['coint', best_freq]*100, 2)}%")
print(f"and the corr is {round(tables[best_pair].at['corr', best_freq],3)}")
print("===========================================")

cointncorr = best_pair.split("_"), best_freq

with open('result/cointncorr.pickle', 'wb') as pk:
    pickle.dump(cointncorr, pk)

Best trading pairs shall be: BTCEUR_BTCGBP under 1m interval
the coint is 56.67%
and the corr is 0.876
